In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [3]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')

In [4]:
df_train.head()  #784 features and 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_features = df_train.iloc[:,1:785]
df_label = df_train.iloc[:,0]

X_test = df_test.iloc[:,0:784]
print(X_test.shape)


(28000, 784)


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(df_features,df_label,test_size=0.2,random_state=10)


In [7]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(33600, 784)
(8400, 784)
(28000, 784)


In [8]:
print((min(X_train.iloc[1]), max(X_train.iloc[1])))

(0, 255)


In [9]:
X_train = X_train.astype('float32')
X_val= X_val.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255; X_val /= 255; X_test /= 255

In [10]:
y_train


3698     4
13466    7
21701    7
29814    5
17325    8
        ..
40059    4
28017    2
29199    5
40061    6
17673    0
Name: label, Length: 33600, dtype: int64

In [11]:
# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_val = keras.utils.to_categorical(y_val, num_digits)


In [12]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 4
print(y_train[3]) # 5

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [13]:
# Starting Model fitting

# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [14]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [15]:
# our model has input layers=1 hidden layers=4 output layer=1 
# Total layers=6

model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate|

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
________________________________________________

In [16]:
# inserting hyperparameters

learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

# we are using  Stochastic Gradient Descent

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [18]:
hist1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_val, y_val))

Epoch 1/20
336/336 - 2s - loss: 1.7062 - accuracy: 0.5672 - val_loss: 0.8432 - val_accuracy: 0.7820
Epoch 2/20
336/336 - 1s - loss: 0.5811 - accuracy: 0.8371 - val_loss: 0.4579 - val_accuracy: 0.8676
Epoch 3/20
336/336 - 1s - loss: 0.3988 - accuracy: 0.8842 - val_loss: 0.3666 - val_accuracy: 0.8950
Epoch 4/20
336/336 - 1s - loss: 0.3361 - accuracy: 0.9022 - val_loss: 0.3200 - val_accuracy: 0.9088
Epoch 5/20
336/336 - 1s - loss: 0.3006 - accuracy: 0.9127 - val_loss: 0.2901 - val_accuracy: 0.9177
Epoch 6/20
336/336 - 1s - loss: 0.2735 - accuracy: 0.9204 - val_loss: 0.2668 - val_accuracy: 0.9219
Epoch 7/20
336/336 - 1s - loss: 0.2517 - accuracy: 0.9261 - val_loss: 0.2575 - val_accuracy: 0.9249
Epoch 8/20
336/336 - 1s - loss: 0.2341 - accuracy: 0.9314 - val_loss: 0.2348 - val_accuracy: 0.9317
Epoch 9/20
336/336 - 1s - loss: 0.2179 - accuracy: 0.9363 - val_loss: 0.2274 - val_accuracy: 0.9320
Epoch 10/20
336/336 - 1s - loss: 0.2043 - accuracy: 0.9405 - val_loss: 0.2081 - val_accuracy: 0.9390

In [19]:
#Achieved a training score of around >98% and a test score of around >96%

#lets try out with different optimization technique

In [20]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_val, y_val))

Epoch 1/20
336/336 - 2s - loss: 0.3350 - accuracy: 0.8992 - val_loss: 0.1517 - val_accuracy: 0.9554
Epoch 2/20
336/336 - 2s - loss: 0.1247 - accuracy: 0.9618 - val_loss: 0.1185 - val_accuracy: 0.9635
Epoch 3/20
336/336 - 2s - loss: 0.0842 - accuracy: 0.9740 - val_loss: 0.1016 - val_accuracy: 0.9683
Epoch 4/20
336/336 - 1s - loss: 0.0580 - accuracy: 0.9819 - val_loss: 0.0917 - val_accuracy: 0.9721
Epoch 5/20
336/336 - 2s - loss: 0.0467 - accuracy: 0.9851 - val_loss: 0.1079 - val_accuracy: 0.9674
Epoch 6/20
336/336 - 2s - loss: 0.0361 - accuracy: 0.9883 - val_loss: 0.1068 - val_accuracy: 0.9706
Epoch 7/20
336/336 - 1s - loss: 0.0296 - accuracy: 0.9901 - val_loss: 0.1279 - val_accuracy: 0.9646
Epoch 8/20
336/336 - 2s - loss: 0.0279 - accuracy: 0.9910 - val_loss: 0.1264 - val_accuracy: 0.9669
Epoch 9/20
336/336 - 1s - loss: 0.0231 - accuracy: 0.9932 - val_loss: 0.1156 - val_accuracy: 0.9701
Epoch 10/20
336/336 - 2s - loss: 0.0174 - accuracy: 0.9947 - val_loss: 0.0939 - val_accuracy: 0.9764

In [22]:
#Achieved a training score of around 99.5% and a test score of around 97.5%

In [23]:
# Now we will use Dropouts to prevent overfitting

Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [24]:
model3 = Model(Inp, output)
model3.summary() # We have 297,910 parameters to estimate

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)              

In [25]:
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
history3 = model3.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_val, y_val))

Epoch 1/20
336/336 [==============================] - 2s 6ms/step - loss: 0.5838 - accuracy: 0.8100 - val_loss: 0.1822 - val_accuracy: 0.9454
Epoch 2/20
336/336 [==============================] - 2s 5ms/step - loss: 0.2294 - accuracy: 0.9326 - val_loss: 0.1349 - val_accuracy: 0.9607
Epoch 3/20
336/336 [==============================] - 2s 5ms/step - loss: 0.1694 - accuracy: 0.9514 - val_loss: 0.1165 - val_accuracy: 0.9644
Epoch 4/20
336/336 [==============================] - 2s 5ms/step - loss: 0.1412 - accuracy: 0.9587 - val_loss: 0.1019 - val_accuracy: 0.9682
Epoch 5/20
336/336 [==============================] - 2s 5ms/step - loss: 0.1224 - accuracy: 0.9636 - val_loss: 0.1020 - val_accuracy: 0.9711
Epoch 6/20
336/336 [==============================] - 2s 5ms/step - loss: 0.1106 - accuracy: 0.9683 - val_loss: 0.0955 - val_accuracy: 0.9705
Epoch 7/20
336/336 [==============================] - 2s 5ms/step - loss: 0.0972 - accuracy: 0.9719 - val_loss: 0.0809 - val_accuracy: 0.9773
Epoch 

In [27]:
# Achieved val accuracy close to 98%
# lets run this model with test data

In [28]:
test_pred = pd.DataFrame(model3.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [29]:
test_pred.to_csv('mnist_submission.csv', index = False)